Bước 1: Phân tích dữ liệu thô (EDA – ở mức số liệu)

In [4]:
import pandas as pd
import numpy as np


# Load dữ liệu
df = pd.read_csv("../data/raw/all_movies_data_1975_2025.csv")

# Xem 5 dòng đầu
print(df.head())



                                Title  Year Duration        MPA  Rating Votes  \
0                 275. Das Kalte Herz  2016   1h 23m        NaN     6.3    17   
1          280. Seven Women for Satan  1976   1h 22m  Not Rated     4.6   681   
2                             1. Jaws  1975    2h 4m         PG     8.1  690K   
3    2. The Rocky Horror Picture Show  1975   1h 40m          R     7.4  174K   
4  3. One Flew Over the Cuckoo's Nest  1975   2h 13m          R     8.7  1.1M   

   méta_score                                        description  \
0         NaN  A poor and lonely coal seller trades his heart...   
1         NaN  A businessman, who's a descendant of a brutal ...   
2        87.0  When a massive killer shark unleashes chaos on...   
3        65.0  A newly-engaged couple have a breakdown in an ...   
4        84.0  In the Fall of 1963, a Korean War veteran and ...   

                                          Movie Link  \
0  https://www.imdb.com/title/tt4724210/?ref_=sr

In [7]:
# Thông tin tổng quan
print("\n=== INFO ===")
df.info()


=== INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30552 entries, 0 to 30551
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  30552 non-null  object 
 1   Year                   30552 non-null  int64  
 2   Duration               30204 non-null  object 
 3   MPA                    23378 non-null  object 
 4   Rating                 30218 non-null  float64
 5   Votes                  30218 non-null  object 
 6   méta_score             14427 non-null  float64
 7   description            30213 non-null  object 
 8   Movie Link             30552 non-null  object 
 9   writers                30552 non-null  object 
 10  directors              30552 non-null  object 
 11  stars                  30320 non-null  object 
 12  budget                 11524 non-null  object 
 13  opening_weekend_Gross  16542 non-null  object 
 14  grossWorldWWide        19400 non-null  o

In [8]:
# Thống kê mô tả numeric
print("\n=== DESCRIBE ===")
print(df.describe())


=== DESCRIBE ===
               Year        Rating    méta_score
count  30552.000000  30218.000000  14427.000000
mean    2000.018297      6.131150     58.007763
std       14.725548      1.167326     17.142367
min     1975.000000      1.100000      1.000000
25%     1987.000000      5.400000     46.000000
50%     2000.000000      6.300000     59.000000
75%     2013.000000      7.000000     71.000000
max     2025.000000     10.000000    100.000000


In [9]:

#Kiểm tra giá trị khuyết (Missing values)
missing_values = df.isnull().sum().sort_values(ascending=False)
print("Giá trị khuyết thiếu:\n", missing_values)



Giá trị khuyết thiếu:
 budget                   19028
méta_score               16125
opening_weekend_Gross    14010
gross_US_Canada          12318
awards_content           11478
grossWorldWWide          11152
MPA                       7174
filming_locations         6493
production_company        1125
release_date               381
Duration                   348
description                339
Rating                     334
Votes                      334
stars                      232
countries_origin            88
Year                         0
Title                        0
directors                    0
writers                      0
Movie Link                   0
genres                       0
Languages                    0
dtype: int64


Nhận xét: Kết quả cho thấy tập dữ liệu tồn tại nhiều giá trị khuyết, đặc biệt ở các biến liên quan đến doanh thu (budget, opening_weekend_Gross, grossWorldWWide, gross_US_Canada) và meta_score. Nguyên nhân chủ yếu do dữ liệu phim cổ điển không được ghi nhận đầy đủ trên IMDb.

In [18]:
#kiểm tra outlier
numeric_cols = [
    "Rating",
    "Votes",
    "grossWorldWWide"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series[(series < lower_bound) | (series > upper_bound)]


for col in numeric_cols:
    outliers = detect_outliers_iqr(df[col].dropna())
    print(f"Số lượng outliers trong {col}: {len(outliers)}")
    
    if len(outliers) > 0:
        print("Một số outliers mẫu:")
        print(outliers.head())
    print("-" * 50)



Số lượng outliers trong Rating: 331
Một số outliers mẫu:
95     2.8
164    2.7
168    2.8
227    2.9
247    2.9
Name: Rating, dtype: float64
--------------------------------------------------
Số lượng outliers trong Votes: 0
--------------------------------------------------
Số lượng outliers trong grossWorldWWide: 2922
Một số outliers mẫu:
2      477916625.0
3      115827018.0
4      109115366.0
5       50004527.0
602    117253345.0
Name: grossWorldWWide, dtype: float64
--------------------------------------------------


Bước 2: Tiển xử lý dữ liệu

In [ ]:
import pandas as pd
import numpy as np

#load data
df = pd.read_csv("../data/raw/all_movies_data_1975_2025.csv")
initial_rows = df.shape[0]
print(f"Số dòng ban đầu: {initial_rows}")

#chuẩn hóa tên cột
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df = df.rename(columns={"méta_score": "meta_score"})

# NHÓM BIẾN
money_cols = [
    "budget", "opening_weekend_gross",
    "grossworldwwide", "gross_us_canada"
]

categorical_cols = [
    "mpa", "countries_origin", "production_company",
    "genres", "languages", "stars"
]

text_cols = [
    "description", "awards_content",
    "filming_locations"
]

# XỬ LÝ VOTES (K, M)
def convert_votes(v):
    if pd.isna(v):
        return np.nan
    v = str(v).upper()
    if "K" in v:
        return float(v.replace("K", "")) * 1_000
    if "M" in v:
        return float(v.replace("M", "")) * 1_000_000
    return float(v)

df["votes"] = df["votes"].apply(convert_votes)


# XỬ LÝ CỘT TIỀN TỆ
for col in money_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r"[^\d.]", "", regex=True)
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ÉP KIỂU NUMERIC
for col in ["year", "rating", "meta_score"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# DROP BIẾN CỐT LÕI
df = df.dropna(subset=["year", "rating"])

# XỬ LÝ MISSING VALUES
numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

for col in text_cols:
    df[col] = df[col].fillna("No information")

# LOG TRANSFORM (GIỮ CỘT GỐC)
log_cols = [
    "votes", "budget",
    "opening_weekend_gross",
    "grossworldwwide", "gross_us_canada"
]

for col in log_cols:
    df[col] = df[col].clip(lower=0)
    df[f"{col}_log"] = np.log1p(df[col])


# KIỂM TRA SAU XỬ LÝ
final_rows = df.shape[0]
print(f"Số dòng sau tiền xử lý: {final_rows}")
print(f"Số dòng bị loại bỏ: {initial_rows - final_rows}")

print("\nGiá trị khuyết sau xử lý:")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# LƯU FILE
output_path = "../data/processed/clean_movies_data_1975_2025.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Đã lưu dữ liệu đã làm sạch tại: {output_path}")

print("\n10 dòng đầu sau tiền xử lý:")
print(df.head(10))


Số dòng ban đầu: 30552
Số dòng sau tiền xử lý: 30218
Số dòng bị loại bỏ: 334

Giá trị khuyết sau xử lý:
release_date    371
duration        171
year              0
title             0
rating            0
votes             0
meta_score        0
description       0
movie_link        0
writers           0
dtype: int64
Đã lưu dữ liệu đã làm sạch tại: ../data/processed/clean_movies_data_1975_2025.csv

10 dòng đầu sau tiền xử lý:
                                title  year duration        mpa  rating  \
0                 275. Das Kalte Herz  2016   1h 23m    Unknown     6.3   
1          280. Seven Women for Satan  1976   1h 22m  Not Rated     4.6   
2                             1. Jaws  1975    2h 4m         PG     8.1   
3    2. The Rocky Horror Picture Show  1975   1h 40m          R     7.4   
4  3. One Flew Over the Cuckoo's Nest  1975   2h 13m          R     8.7   
5                4. Dog Day Afternoon  1975    2h 5m          R     8.0   
6                          5. Shampoo  1975   1

Tách Data - 80 train - 20 test

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv("../data/processed/clean_movies_data_1975_2025.csv")

print(f"Shape full dataset: {df.shape}")


#spit train/test
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

print(f"Train shape: {train_df.shape}")
print(f"Test shape : {test_df.shape}")

#lưu file
train_path = "../data/split/train_raw.csv"
test_path  = "../data/split/test_raw.csv"

train_df.to_csv(train_path, index=False, encoding="utf-8-sig")
test_df.to_csv(test_path, index=False, encoding="utf-8-sig")

print("Split data completed!")
print(f"Train raw saved to: {train_path}")
print(f"Test raw saved to : {test_path}")


Shape full dataset: (30218, 28)
Train shape: (24174, 28)
Test shape : (6044, 28)
Split data completed!
Train raw saved to: ../data/split/train_raw.csv
Test raw saved to : ../data/split/test_raw.csv


In [ ]:
#đặt feature ngay sau đoạn split train/test nha

# Linear models 
linear_features = [
    "budget",
    "runtime",
    "vote_average",
    "vote_count",
    "popularity"
]

# Tree-based models (Random Forest)
tree_features = [
    "budget",
    "runtime",
    "vote_average",
    "vote_count",
    "popularity",
    "release_year",
    "num_genres"
]

# KNN models
knn_features = [
    "budget",
    "runtime",
    "vote_average",
    "popularity"
]
